In [1]:
import os, json
import numpy as np
import ceruleanml.data as data
from pathlib import Path
import skimage.io as skio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from icevision.all import *
import icedata
import skimage
import pandas as pd

ml_data_path = os.path.join(os.path.abspath(os.getcwd()),"../../data/cv2_transfer/outputs/skytruth_annotation/first_phase/")
path = Path(ml_data_path)
class_folders = list(path.glob("*/"))

with open("../../data/cv2_transfer/instances_slicks_test_v2.json") as annotation:
    data_js=json.load(annotation)

In [2]:
class_map = {
    "Infrastructure": 1,
    "Natural Seep": 2,
    "Coincident Vessel": 3,
    "Recent Vessel": 4,
    "Old Vessel": 5,
    "Ambiguous": 6,
    "Hard Negatives": 0,
}

class_map

{'Infrastructure': 1,
 'Natural Seep': 2,
 'Coincident Vessel': 3,
 'Recent Vessel': 4,
 'Old Vessel': 5,
 'Ambiguous': 6,
 'Hard Negatives': 0}

In [3]:
list(data_js['categories'])

[{'supercategory': 'slick', 'id': 1, 'name': 'infra_slick'},
 {'supercategory': 'slick', 'id': 2, 'name': 'natural_seep'},
 {'supercategory': 'slick', 'id': 3, 'name': 'coincident_vessel'},
 {'supercategory': 'slick', 'id': 4, 'name': 'recent_vessel'},
 {'supercategory': 'slick', 'id': 5, 'name': 'old_vessel'},
 {'supercategory': 'slick', 'id': 6, 'name': 'ambiguous'}]

In [4]:
infra_slick_stats = [] 
natural_seep_stats = []
coincident_vessel_stats = []
recent_vessel_stats = []
old_vessel_stats = []
ambiguous_stats = []

In [5]:
for c in list(data_js['annotations']):
    cat = c['category_id']
    if cat == 1:
        infra_slick_stats.append(c)
    elif cat == 2:
        natural_seep_stats.append(c)
    elif cat == 3:
        coincident_vessel_stats.append(c)
    elif cat == 4:
        recent_vessel_stats.append(c)
    elif cat == 5:
        old_vessel_stats.append(c)
    elif cat == 6:
        ambiguous_stats.append(c)                          

In [15]:
parser = parsers.COCOMaskParser(annotations_filepath="../../data/cv2_transfer/instances_slicks_test_v2.json", img_dir="../../data/tiled_image_slicks_test_v2")

In [16]:
train_records = parser.parse(data_splitter=data.SingleSplitSplitter())
record = train_records[0][0]

  0%|          | 0/487 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/487 [00:00<?, ?it/s]

In [60]:
infra_slick_stats_area_segment = [] 
natural_seep_stats_area_segment = []
coincident_vessel_stats_area_segment = []
recent_vessel_stats_area_segment = []
old_vessel_stats_area_segment = []
ambiguous_stats_area_segment = []

infra_slick_stats_area_bbox = [] 
natural_seep_stats_area_bbox = []
coincident_vessel_stats_area_bbox = []
recent_vessel_stats_area_bbox = []
old_vessel_stats_area_bbox = []
ambiguous_stats_area_bbox = []

infra_slick_stats_major_axis_length_segment = [] 
natural_seep_stats_major_axis_length_segment = []
coincident_vessel_stats_major_axis_length_segment = []
recent_vessel_stats_major_axis_length_segment = []
old_vessel_stats_major_axis_length_segment = []
ambiguous_stats_major_axis_length_segment = []

infra_slick_stats_minor_axis_length_segment = [] 
natural_seep_stats_minor_axis_length_segment = []
coincident_vessel_stats_minor_axis_length_segment = []
recent_vessel_stats_minor_axis_length_segment = []
old_vessel_stats_minor_axis_length_segment = []
ambiguous_stats_minor_axis_length_segment = []

In [ ]:
def get_minor_axis_length_proxy(bbox):
    ml1 = bbox[2]-bbox[0]
    ml2 = bbox[3]-bbox[1]
    ml3 = min(ml1, ml2)
    return ml3

In [40]:
def get_props_segments(d, category_list, category_prop_area_seg_list, category_prop_area_bbox_list, category_major_axis_length_list, category_minor_axis_length_list):
    mask_array = d['detection']['masks'][0].to_mask(d['common']['height'],d['common']['width']).data
    rprops = skimage.measure.regionprops(mask_array.transpose(1,2,0))
    category_prop_area_seg_list.append(rprops[0]['area'])
    category_prop_area_bbox_list.append(rprops[0]['area_bbox'])
    category_major_axis_length_list.append(rprops[0]['axis_major_length']) #major_axis_length'])
    try:
        category_minor_axis_length_list.append(rprops[0]['axis_minor_length'])
    except:
        axis_minor_length_proxy = get_minor_axis_length_proxy(rprops[0]['bbox'])
        category_minor_axis_length_list.append(axis_minor_length_proxy) 
        print("minor axis length math domain error for, so using proxy: ", d['common']['filepath'])

axis_major_length, axis_minor_length, area, and area_bbox

In [ ]:
for i in range(len(train_records[0])):
    record = train_records[0][i]
    d = record.as_dict()
    if d['detection']['labels'] == ['infra_slick']:
        get_props_segments(d, infra_slick_stats, infra_slick_stats_area_segment, infra_slick_stats_area_bbox, infra_slick_stats_major_axis_length_segment, infra_slick_stats_minor_axis_length_segment)
    elif d['detection']['labels'] == ['natural_seep']:
        get_props_segments(d, natural_seep_stats, natural_seep_stats_area_segment, natural_seep_stats_area_bbox, natural_seep_stats_major_axis_length_segment, natural_seep_stats_minor_axis_length_segment)
    elif d['detection']['labels'] == ['coincident_vessel']:
        get_props_segments(d, coincident_vessel_stats, coincident_vessel_stats_area_segment, coincident_vessel_stats_area_bbox, coincident_vessel_stats_major_axis_length_segment, coincident_vessel_stats_minor_axis_length_segment)
    elif d['detection']['labels'] == ['recent_vessel']:
        get_props_segments(d, recent_vessel_stats, recent_vessel_stats_area_segment, recent_vessel_stats_area_bbox, recent_vessel_stats_major_axis_length_segment, recent_vessel_stats_minor_axis_length_segment)
    elif d['detection']['labels'] == ['old_vessel']:
        get_props_segments(d, old_vessel_stats, old_vessel_stats_area_segment, old_vessel_stats_area_bbox, old_vessel_stats_major_axis_length_segment, old_vessel_stats_minor_axis_length_segment)
    elif d['detection']['labels'] == ['ambiguous']:
        get_props_segments(d, ambiguous_stats, ambiguous_stats_area_segment, ambiguous_stats_area_bbox, ambiguous_stats_major_axis_length_segment, ambiguous_stats_minor_axis_length_segment)

In [42]:
max_infra_slick_stats_area_bbox, avg_infra_slick_stats_area_bbox = max(infra_slick_stats_area_bbox), np.mean((infra_slick_stats_area_bbox))
max_infra_slick_stats_area_segment, avg_infra_slick_stats_area_segment = max(infra_slick_stats_area_segment), np.mean((infra_slick_stats_area_segment))
max_infra_slick_stats_major_axis_length_segment, avg_infra_slick_stats_major_axis_length_segment = max(infra_slick_stats_major_axis_length_segment), np.mean((infra_slick_stats_major_axis_length_segment))
max_infra_slick_stats_minor_axis_length_segment, avg_infra_slick_stats_minor_axis_length_segment = max(infra_slick_stats_minor_axis_length_segment), np.mean((infra_slick_stats_minor_axis_length_segment))

max_natural_seep_stats_area_bbox, avg_natural_seep_stats_area_bbox = max(natural_seep_stats_area_bbox), np.mean((natural_seep_stats_area_bbox))
max_natural_seep_stats_area_segment, avg_natural_seep_stats_area_segment = max(natural_seep_stats_area_segment), np.mean((natural_seep_stats_area_segment))
max_natural_seep_stats_major_axis_length_segment, avg_natural_seep_stats_major_axis_length_segment = max(natural_seep_stats_major_axis_length_segment), np.mean((natural_seep_stats_major_axis_length_segment))
max_natural_seep_stats_minor_axis_length_segment, avg_natural_seep_stats_minor_axis_length_segment = max(natural_seep_stats_minor_axis_length_segment), np.mean((natural_seep_stats_minor_axis_length_segment))

max_coincident_vessel_stats_area_bbox, avg_coincident_vessel_stats_area_bbox = max(coincident_vessel_stats_area_bbox), np.mean((coincident_vessel_stats_area_bbox))
max_coincident_vessel_stats_area_segment, avg_coincident_vessel_stats_area_segment = max(coincident_vessel_stats_area_segment), np.mean((coincident_vessel_stats_area_segment))
max_coincident_vessel_stats_major_axis_length_segment, avg_coincident_vessel_stats_major_axis_length_segment = max(coincident_vessel_stats_major_axis_length_segment), np.mean((coincident_vessel_stats_major_axis_length_segment))
max_coincident_vessel_stats_minor_axis_length_segment, avg_coincident_vessel_stats_minor_axis_length_segment = max(coincident_vessel_stats_minor_axis_length_segment), np.mean((coincident_vessel_stats_minor_axis_length_segment))

max_recent_vessel_stats_area_bbox, avg_recent_vessel_stats_area_bbox = max(recent_vessel_stats_area_bbox), np.mean((recent_vessel_stats_area_bbox))
max_recent_vessel_stats_area_segment, avg_recent_vessel_stats_area_segment = max(recent_vessel_stats_area_segment), np.mean((recent_vessel_stats_area_segment))
max_recent_vessel_stats_major_axis_length_segment, avg_recent_vessel_stats_major_axis_length_segment = max(recent_vessel_stats_major_axis_length_segment), np.mean((recent_vessel_stats_major_axis_length_segment))
max_recent_vessel_stats_minor_axis_length_segment, avg_recent_vessel_stats_minor_axis_length_segment = max(recent_vessel_stats_minor_axis_length_segment), np.mean((recent_vessel_stats_minor_axis_length_segment))

max_old_vessel_stats_area_bbox, avg_old_vessel_stats_area_bbox = max(old_vessel_stats_area_bbox), np.mean((old_vessel_stats_area_bbox))
max_old_vessel_stats_area_segment, avg_old_vessel_stats_area_segment = max(old_vessel_stats_area_segment), np.mean((old_vessel_stats_area_segment))
max_old_vessel_stats_major_axis_length_segment, avg_old_vessel_stats_major_axis_length_segment = max(old_vessel_stats_major_axis_length_segment), np.mean((old_vessel_stats_major_axis_length_segment))
max_old_vessel_stats_minor_axis_length_segment, avg_old_vessel_stats_minor_axis_length_segment = max(old_vessel_stats_minor_axis_length_segment), np.mean((old_vessel_stats_minor_axis_length_segment))

max_ambiguous_stats_area_bbox, avg_ambiguous_stats_area_bbox = max(ambiguous_stats_area_bbox), np.mean((ambiguous_stats_area_bbox))
max_ambiguous_stats_area_segment, avg_ambiguous_stats_area_segment = max(ambiguous_stats_area_segment), np.mean((ambiguous_stats_area_segment))
max_ambiguous_stats_major_axis_length_segment, avg_ambiguous_stats_major_axis_length_segment = max(ambiguous_stats_major_axis_length_segment), np.mean((ambiguous_stats_major_axis_length_segment))
max_ambiguous_stats_minor_axis_length_segment, avg_ambiguous_stats_minor_axis_length_segment = max(ambiguous_stats_minor_axis_length_segment), np.mean((ambiguous_stats_minor_axis_length_segment))



In [67]:
class_list = [1,2,3,4,5,6]
class_list_string = ['Infrastructure', 'Natural Seep', 'Coincident Vessel', 'Recent Vessel', 'Old Vessel', 'Ambiguous']

In [62]:
avg_area_segment_list = [avg_infra_slick_stats_area_segment, avg_natural_seep_stats_area_segment, avg_coincident_vessel_stats_area_segment, avg_recent_vessel_stats_area_segment, avg_old_vessel_stats_area_segment, avg_ambiguous_stats_area_segment]

In [63]:
avg_area_bbox_list = [avg_infra_slick_stats_area_bbox, avg_natural_seep_stats_area_bbox, avg_coincident_vessel_stats_area_bbox, avg_recent_vessel_stats_area_bbox, avg_old_vessel_stats_area_bbox, avg_ambiguous_stats_area_bbox]

In [64]:
avg_major_axis_length_segment_list = [avg_infra_slick_stats_major_axis_length_segment, avg_natural_seep_stats_major_axis_length_segment, avg_coincident_vessel_stats_major_axis_length_segment, avg_recent_vessel_stats_major_axis_length_segment,avg_old_vessel_stats_major_axis_length_segment, avg_ambiguous_stats_major_axis_length_segment]   

In [65]:
avg_minor_axis_length_segment_list = [avg_infra_slick_stats_minor_axis_length_segment, avg_natural_seep_stats_minor_axis_length_segment, avg_coincident_vessel_stats_minor_axis_length_segment, avg_recent_vessel_stats_minor_axis_length_segment,avg_old_vessel_stats_minor_axis_length_segment, avg_ambiguous_stats_minor_axis_length_segment] 

In [68]:
stats_df = pd.DataFrame(
    {'classes (integer)': class_list,
     'classes (string)': class_list_string,
     'avg area of segment': avg_area_segment_list,
     'avg area of bbox': avg_area_bbox_list,
     'avg major axis length of segment': avg_major_axis_length_segment_list,
     'avg minor axis length of segment': avg_minor_axis_length_segment_list
    })

In [ ]:
stats_df